In [1]:
!pip3 install --quiet --upgrade git+https://github.com/hpe-container-platform-community/hpecp-client@master

In [2]:
from hpecp import ContainerPlatformClient, APIException
from hpecp.k8s_cluster import K8sClusterHostConfig
import os, sys

hpeclient = ContainerPlatformClient(username='admin', 
                                password='admin123', 
                                api_host='127.0.0.1', 
                                api_port=8080,
                                use_ssl=True,
                                verify_ssl='/certs/hpecp-ca-cert.pem')

hpeclient.create_session()

print("\nHPE Container Platform K8S Clusters:\n")
print(hpeclient.k8s_cluster.list().tabulate())


HPE Container Platform K8S Clusters:

+-----------------------+------+-------------+-------------+--------------------+----------------------+--------------+--------+
|          id           | name | description | k8s_version | created_by_user_id | created_by_user_name | created_time | status |
+-----------------------+------+-------------+-------------+--------------------+----------------------+--------------+--------+
| /api/v2/k8scluster/20 | def  | my cluster  |   1.17.0    |   /api/v1/user/5   |        admin         |  1588260014  | ready  |
+-----------------------+------+-------------+-------------+--------------------+----------------------+--------------+--------+


In [3]:
from kubernetes import client, config
import tempfile

cluster_list = hpeclient.k8s_cluster.list()
if len(cluster_list) == 0:
    print("No clusters found.")
else:
    # get the kube admin config for the first cluster 
    admin_kube_config = cluster_list[0].admin_kube_config

    #  write it to a temporary file
    with tempfile.NamedTemporaryFile(mode = "w") as fp:
        fp.write(admin_kube_config)
        fp.flush()

        # read the kube config with the kubernetes client
        config.load_kube_config(fp.name)

        # list the pods        
        v1 = client.CoreV1Api()
        print("\nListing pods with their IPs:\n")
        ret = v1.list_pod_for_all_namespaces(watch=False)
        for i in ret.items:
            print("{:>12}   {:>20}   {}".format(i.status.pod_ip, i.metadata.namespace, i.metadata.name))



Listing pods with their IPs:

10.192.0.144                  hpecp   hpecp-agent-8499d5798b-z7xf5
   10.1.0.50                  hpecp   hpecp-fsmount-6mp4p
  10.1.0.181                  hpecp   hpecp-fsmount-snw66
10.192.0.147                  hpecp   kubedirector-7fdbdb7775-w7q6s
   10.1.0.50            kube-system   canal-2px9j
  10.1.0.181            kube-system   canal-nswzl
  10.192.0.9            kube-system   coredns-6955765f44-597tp
 10.192.0.10            kube-system   coredns-6955765f44-w75qj
  10.1.0.181            kube-system   etcd-ip-10-1-0-181.eu-west-2.compute.internal
  10.1.0.181            kube-system   kube-apiserver-ip-10-1-0-181.eu-west-2.compute.internal
  10.1.0.181            kube-system   kube-controller-manager-ip-10-1-0-181.eu-west-2.compute.internal
   10.1.0.50            kube-system   kube-proxy-472fz
  10.1.0.181            kube-system   kube-proxy-lrk8s
  10.1.0.181            kube-system   kube-scheduler-ip-10-1-0-181.eu-west-2.compute.internal
10.192.